In [25]:
#Solve Numbers 

from itertools import permutations
from operator import add, sub, mul, truediv

def solve_numbers(numbers, target):
    operations = [add, sub, mul, truediv]
    op_symbols = {add: '+', sub: '-', mul: '*', truediv: '/'}
    best_solution = None

    def backtrack(current_value, index, expression):
        if current_value == target:
            return expression
        if index == len(numbers):
            return None
        
        for i in range(len(operations)):
            if index < len(numbers):
                next_value = numbers[index]
                if operations[i] == truediv and next_value == 0:
                    continue  # Skip division by zero
                new_value = operations[i](current_value, next_value)
                new_expression = f"({expression} {op_symbols[operations[i]]} {next_value})"
                result = backtrack(new_value, index + 1, new_expression)
                if result:
                    return result
        return None

    for perm in permutations(numbers):
        result = backtrack(perm[0], 1, str(perm[0]))
        if result:
            return result
    
    return "No valid solution found"

    



In [26]:
import random
import operator

# Define operations and their corresponding functions
operations = {
    '+': operator.add,
    '-': operator.sub,
    '*': operator.mul,
    '/': operator.truediv
}

def random_expression(numbers, operations):
    expr = [random.choice(numbers)]  # Start with a random number
    for _ in range(3):  # Generate expression with 3 operations
        operation = random.choice(list(operations.keys()))
        number = random.choice(numbers)
        expr.append(operation)
        expr.append(number)
    return expr

def evaluate_expression(expr):
    result = expr[0]
    for i in range(1, len(expr), 2):
        operation = operations[expr[i]]
        next_value = expr[i+1]
        if operation == operations['/'] and next_value == 0:
            return float('inf')  # Avoid division by zero
        result = operation(result, next_value)
    return result

def fitness(expr, target):
    result = evaluate_expression(expr)
    if result == float('inf'):
        return 0  # Penalize for invalid operations
    return 1 / (1 + abs(result - target)**2)  # Use square to emphasize closer values

def mutate(expr, numbers, operations):
    idx = random.randint(0, len(expr) - 1)
    if idx % 2 == 0:  # Mutate number
        expr[idx] = random.choice(numbers)
    else:  # Mutate operation
        expr[idx] = random.choice(list(operations.keys()))
    return expr

def crossover(parent1, parent2):
    point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    return parent1[:point] + parent2[point:]

def genetic_algorithm(numbers, target, max_generations=100):
    population = [random_expression(numbers, operations) for _ in range(50)]
    for generation in range(max_generations):
        population = sorted(population, key=lambda x: -fitness(x, target))[:2] + population
        new_population = []
        while len(new_population) < len(population):
            parent1, parent2 = random.sample(population, 2)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            new_population.extend([mutate(child1, numbers, operations), mutate(child2, numbers, operations)])
        population = new_population
        best_solution = sorted(population, key=lambda x: -fitness(x, target))[0]
        print(f"Generation {generation}: Best Value = {evaluate_expression(best_solution)}, Best Fitness = {fitness(best_solution, target)}")
        if abs(evaluate_expression(best_solution) - target) <= 10:
            print("Acceptable solution found.")
            break
    return best_solution


In [27]:
# Tests 
import unittest

class TestSolveNumbersAndGeneticAlgorithm(unittest.TestCase):
    def test_solve_numbers_basic(self):
        numbers = [25, 10, 2, 8, 1, 5]
        target = 100
        result = solve_numbers(numbers, target)
        self.assertIsNotNone(result)
        self.assertEqual(eval(result), target)

    def test_solve_numbers_no_solution(self):
        numbers = [1, 3, 5]
        target = 100
        result = solve_numbers(numbers, target)
        self.assertEqual(result, "No valid solution found")

    def test_genetic_algorithm_basic(self):
        numbers = [25, 50, 75, 100, 3, 6]
        target = 952
        solution = genetic_algorithm(numbers, target)
        solution_value = evaluate_expression(solution)
        self.assertTrue(abs(solution_value - target) <= 10)

    def test_genetic_algorithm_handles_zero(self):
        numbers = [0, 25, 50, 75]
        target = 10
        solution = genetic_algorithm(numbers, target)
        solution_value = evaluate_expression(solution)
        self.assertTrue(abs(solution_value - target) <= 10)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


....


----------------------------------------------------------------------
Ran 4 tests in 0.024s

OK


Generation 0: Best Value = 883.3333333333334, Best Fitness = 0.0002120391094357406
Generation 1: Best Value = 753, Best Fitness = 2.5251249936871874e-05
Generation 2: Best Value = 933.3333333333334, Best Fitness = 0.0028616852146264027
Generation 3: Best Value = 883.3333333333334, Best Fitness = 0.0002120391094357406
Generation 4: Best Value = 719, Best Fitness = 1.841959845275373e-05
Generation 5: Best Value = 1125, Best Fitness = 3.341129301703976e-05
Generation 6: Best Value = 925, Best Fitness = 0.0013698630136986301
Generation 7: Best Value = 1250.0, Best Fitness = 1.1260627216935984e-05
Generation 8: Best Value = 883.3333333333334, Best Fitness = 0.0002120391094357406
Generation 9: Best Value = 897, Best Fitness = 0.00033046926635822867
Generation 10: Best Value = 900.0, Best Fitness = 0.0003696857670979667
Generation 11: Best Value = 1000, Best Fitness = 0.0004338394793926247
Generation 12: Best Value = 833.3333333333334, Best Fitness = 7.100871829263487e-05
Generation 13: Best 